In [1]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
train_df = pd.read_csv("../../input/feedback-prize-english-language-learning/train.csv")
train_df.head(2)

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5


In [3]:
model_path_list = [
    #'../../01_Baseline/exp/result/01_v1_24/oof_df.csv', # deberta-base
    #'../../01_Baseline/exp/result/01_v1_25/oof_df.csv', # deberta-large
    #'../../01_Baseline/exp/result/01_v1_27/oof_df.csv', # deberta-v3-base

    #'../../01_Baseline/exp/result/01_v1_43/oof_df.csv', # deberta-v3-large
    #'../../01_Baseline/exp/result/01_v1_44/oof_df.csv', # roberta-base
    '../../01_Baseline/exp/result/01_v1_45/oof_df.csv', # roberta-large
    
    #'../../01_Baseline/exp/result/01_v1_47/oof_df.csv', # electra-base
    #'../../01_Baseline/exp/result/01_v1_49/oof_df.csv', # deepset/roberta-base-squad2
    #'../../01_Baseline/exp/result/01_v1_50/oof_df.csv', # distilroberta-base
    
    #'../../01_Baseline/exp/result/01_v1_54/oof_df.csv', # electra-large
    #'../../01_Baseline/exp/result/01_v1_55/oof_df.csv', # deepset/roberta-large-squad2
    #'../../01_Baseline/exp/result/01_v1_56/oof_df.csv', # xlm-roberta-large
    #'../../01_Baseline/exp/result/01_v1_59/oof_df.csv', # bert-large-cased-whole-word-masking
    #'../../01_Baseline/exp/result/01_v1_60/oof_df.csv', # bert-large-cased
    '../../07_Embeddings/exp/result/07_v1_01/oof_df_svr_notebook_v03.csv', # svr
    
    '../../11_Pseudo2/exp/result/11_v1_02/oof_df.csv', # deberta-v3-base
    '../../11_Pseudo2/exp/result/11_v1_03/oof_df.csv', # deberta-v3-large
    
]

oof_df_list = [
    pd.read_csv(model_path) for model_path in model_path_list
]

for oof_df in oof_df_list:
    oof_df = train_df[['text_id']].merge(oof_df, how='left', on='text_id')
    
num_models = len(model_path_list)

TARGET_COLS = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']

preds = 0
for oof_df in oof_df_list:
    preds += oof_df[TARGET_COLS].values / num_models

oofs = []
for oof_df in oof_df_list:
    oofs.append(oof_df[TARGET_COLS].values)

In [22]:
def postprocessing(x):
    for v in [1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]:
        if v-0.05 < x and x < v + 0.05:
            return v
    return x

In [23]:
oof_df = oof_df_list[0].copy()
for i,col in enumerate(TARGET_COLS):
    oof_df[col] = [postprocessing(v) for v in preds[:,i]]

In [24]:
import numpy as np

def calc_metric(pred, gt):
    '''
    pred : (num_data, num_labels)
    gt : (num_data, num_labels)
    '''
    score = np.sqrt(np.mean((pred - gt)**2, axis=0))
    score = score.mean()
    return score

In [25]:
score = calc_metric(pred=oof_df[TARGET_COLS].values, gt=train_df[TARGET_COLS].values)
print('CV={:.4f}'.format(score))

CV=0.4460


In [26]:
score = calc_metric(pred=oof_df[TARGET_COLS].values.clip(1,5), gt=train_df[TARGET_COLS].values)
print('CV={:.4f}'.format(score))

CV=0.4460


In [ ]:
ensemble_predictions=np.stack(oofs)

In [ ]:
def ensemble_score(weights,return_pred=False):
    weights=np.array(weights)
    weights=weights.reshape(-1,1,1)/weights.sum()
    p=weights.reshape(-1,1,1)*ensemble_predictions
    p=p.sum(0)
    score=calc_metric(p, train_df[TARGET_COLS])
    if return_pred:
        return score,p
    else:
        return score   

In [ ]:
from skopt import gp_minimize

results=gp_minimize(ensemble_score, np.array([[0.1,1] for i in range(len(ensemble_predictions))]),
                    verbose=True,random_state=2022)

In [ ]:
best_weights=np.array(results['x'])/sum(results['x'])
print(best_weights)

In [ ]:
# smaller is better for this metric
score,ensemble_pred=ensemble_score(best_weights,True)
score

In [ ]:
print('CV={:.4f}'.format(score))